In [1]:
import networkx as nx
import networkx.algorithms.community as nx_comm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
root = r"my_path"

In [44]:
df=pd.read_csv(os.path.join(root, r"edge_list.csv"))
df

,source,target,weight
0,7,9,1
1,7,76,1
2,7,78,1
3,7,109,1
4,7,276,1
...,...,...,...
667501,22581,22637,1
667502,22609,22613,1
667503,22609,22619,1
667504,22613,22619,1


# remove low-reviewer nodes

In [45]:
country_df = pd.read_csv(os.path.join(root, r"country_bipartite.csv"))[["journal_name_wos","reviewer_id"]]
a = country_df.groupby(["journal_name_wos"])["reviewer_id"].nunique().reset_index()
a

,journal_name_wos,reviewer_id
0,1,2
1,7,3
2,9,18
3,12,174
4,17,6
...,...,...
9215,22632,5
9216,22633,1
9217,22637,9
9218,22638,4


In [ ]:
low_degree_list = list(a.loc[a["reviewer_id"] < 5, "journal_name_wos"].unique())

In [ ]:
# export new csv
df[~(df["source"].isin(low_degree_list) | df["target"].isin(low_degree_list))].to_csv(os.path.join(root, r"over10_edge_list.csv"),index=False)

In [48]:
# get node list
g_df = pd.read_csv(os.path.join(root, r"over10_edge_list.csv"))
over_10_nodes = pd.concat([g_df["source"],g_df["target"]]).unique()
len(over_10_nodes)

6095

# add discipline/country attributes

In [49]:
ctry_df = pd.read_csv(os.path.join(root, r"jcr_result.csv"))[["Code_Revue","alpha2_journal","country_group"]]

In [50]:
left = pd.DataFrame(over_10_nodes,columns=["journal_name_wos"])

In [51]:
node_list=pd.merge(left, ctry_df, left_on="journal_name_wos", right_on="Code_Revue", how="left")
node_list = node_list.drop("Code_Revue",axis=1)

In [52]:
node_list.loc[node_list["country_group"].isna(), "country_group"] = -1

In [53]:
disc_df = pd.read_csv(os.path.join(root, r"journal_discipline.csv"))
node_list2=pd.merge(node_list, disc_df, on="journal_name_wos", how="left")

In [54]:
label_df = pd.read_csv(os.path.join(root, r"journal_abbrev.csv"))[["journal_name_wos","Abbrev"]]
label_df = label_df.rename(columns={"Abbrev":"Label"})
node_list3=pd.merge(node_list2, label_df, on="journal_name_wos", how="left")
node_list3 = node_list3.rename(columns={"journal_name_wos": "Id"})

In [ ]:
#node_list3.to_csv(os.path.join(root, r"over10_node_list.csv"), index=False)

# remove nodes without JCR information

In [55]:
node_df = (pd.read_csv(os.path.join(root, r"over10_node_list.csv"))
           .dropna()
          )
node_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6058 entries, 0 to 6094
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              6058 non-null   int64  
 1   alpha2_journal  6058 non-null   object 
 2   country_group   6058 non-null   float64
 3   did             6058 non-null   int64  
 4   d1              6058 non-null   object 
 5   d2              6058 non-null   object 
 6   d3              6058 non-null   object 
 7   totalpubs       6058 non-null   int64  
 8   oapubs          6058 non-null   int64  
 9   oa_rate         6058 non-null   float64
 10  Label           6058 non-null   object 
 11  quartile        6058 non-null   float64
 12  isOA            6058 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 662.6+ KB


In [56]:
ids = node_df["Id"].unique()
edge_df = pd.read_csv(os.path.join(root, r"over10_edge_list.csv"))
edge_df = edge_df.loc[edge_df["source"].isin(ids) & edge_df["target"].isin(ids)]
node_df.to_csv(os.path.join(root, r"over10_node_list_nonull.csv"), index=False)
edge_df.to_csv(os.path.join(root, r"over10_edge_list_nonull.csv"), index=False)